<h2>Project Name: FLAUTO: Federated Learning with Automated
Dual-Level Hyperparameter Tuning  </h2>
<h3> Written by Rakib Ul Haque and supervised by Dr. Panos P. Markopoulos </h3>
<h4> Email: panagiotis.markopoulos@utsa.edu</h4>

<h1><b>Libraries</b></h1>

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from ultralytics import YOLO
import shutil
import os
from IPython.display import clear_output
import time
import pickle
import json
from IPython.display import clear_output
import copy
from copy import deepcopy
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


<h1><b>Measures</b></h1>

In [2]:
# print function
def Print(string, dictionary):
    first_key = next(iter(dictionary))
    first_value = dictionary[first_key]
    print(f"{string}:{first_key}: {first_value[0][0]}\n")

# deleting run folder for saving spaces
def delete_folder(folder_path):
    if os.path.exists(folder_path):
        # Remove the folder and all its contents
        shutil.rmtree(folder_path)
        print(f"Folder '{folder_path}' deleted successfully!")
    else:
        print(f"Folder '{folder_path}' does not exist.")


<h1><b>Train function</b></h1>

In [5]:
def average_updates(w, n_k):
    w_avg = deepcopy(w[0])
    for key in w_avg.keys():
        w_avg[key] = torch.mul(w_avg[key], n_k[0])
        for i in range(1, len(w)):
            w_avg[key] = torch.add(w_avg[key], w[i][key], alpha=n_k[i])
        w_avg[key] = torch.div(w_avg[key], sum(n_k))
    return w_avg

In [6]:
def training(i_w, E, r, c):
    global learning_rate, epochs
    # Initialize the local model
    local_model = YOLO("initial_weights.pt").to(device)
    local_model.load_state_dict(i_w)

    # Perform local training
    local_model.train(data=f"{fl_a}/{set_up}/c{c}.yaml", 
                      project=f"{dst_folder}/train/round_{r}_client_{c}", 
                      workers=0, 
                      epochs=epochs,  # Ensure epochs is an integer
                      imgsz=512, 
                      lr0=learning_rate,  # Ensure learning_rate is float
                      split='train',
                      batch=4, 
                      optimizer=opti,  # Ensure optimizer is correctly specified
                      val=True, device=0, warmup_epochs=0)

    # Collect final weights
    client_final_weights = {k: v.clone().float().to(device) for k, v in local_model.state_dict().items()}
    
    # Calculate weight differences
    # weight_differences = calculate_weight_differences({k: v.float().to(device) for k, v in i_w.items()}, 
                                                      # client_final_weights)
    model_update = {}
    for key in local_model.state_dict():
        model_update[key] = torch.sub(i_w[key], client_final_weights[key])
    
    return model_update


<h1><b>FL structure</b></h1>

In [7]:
def federated_learning(i_w, C, P, R, E, b_size, lr=0.001, tau=1e-3):
    global global_model
    global_model.load_state_dict(i_w)
    G = None

    # m = {k: torch.full_like(v, float(tau), device=device, dtype=torch.float) for k, v in global_model.state_dict().items()}
    # # m = {k: torch.zeros_like(v, device=device, dtype=torch.float) for k, v in global_model.state_dict().items()}  # 1st moment vector
    # v = {k: torch.full_like(v, float(tau**2), device=device, dtype=torch.float) for k, v in global_model.state_dict().items()}  # 2nd moment vector initialized to τ
    # # v = {k: torch.zeros_like(v, device=device, dtype=torch.float) for k, v in global_model.state_dict().items()}  # 2nd moment vector initialized to τ

    for r in range(1, R + 1):
        delta = []

        # Create a copy of the current global model's weights
        #G = {k: v.clone().float() for k, v in global_model.state_dict().items()}
        i_w = {k: v.clone().float() for k, v in global_model.state_dict().items()}
        Print("Model's initial weights:", {k: v.float() for k, v in i_w.items()})

        # Loop for selected clients
        for c in range(1, C + 1):
            # Training on the client
            clients_delta = training(i_w, E, r, c)
            delta.append(clients_delta)

        # Average the gradients from clients
        # average_gradients = average_function(delta)
        # update_avg = average_updates(all_clients_updates, data_size)
        average_gradients = average_updates(delta, data_size)  #average_function(delta)
        
        Print("Weights difference:", average_gradients)

        if G is None:
            G = {key: torch.zeros_like(param).float() for key, param in average_gradients.items()}
            
        for key in i_w:
            # Accumulate the sum of squares of gradients
            G[key] += average_gradients[key] ** 2

            # Update weights using Adagrad rule
            i_w[key] = i_w[key] - ( lr * average_gradients[key] / (torch.sqrt(G[key]) + tau))

        
        global_model.load_state_dict(i_w)

        updated_weights = {k: v.clone().float() for k, v in global_model.state_dict().items()}
        Print(f"Updated global model after round {r}:", {k: v.float() for k, v in updated_weights.items()})

        # Save the updated weights
        os.makedirs(os.path.join(dst_folder, "weights"), exist_ok=True)
        torch.save(global_model, f'{dst_folder}/weights/after_round_{r}_weights.pt')

        # Load the updated weights into the global model for validation
        #global_model = YOLO("initial_weights.pt").to(device)
        #global_model.load_state_dict(updated_weights)
        val_model = YOLO("initial_weights.pt").to(device)
        val_model.load_state_dict(updated_weights)
        
        # c_weight = {k: v.clone().float() for k, v in global_model.state_dict().items()}
        # print(f"Check updated global model after round {r}:", {k: v.float() for k, v in c_weight.items()})

        
        # Perform validation
        validation_results = val_model.val(data=f"{fl_a}/c5.yaml", project=f"{dst_folder}/val/round_{r}", imgsz=512, batch=4, split='val', workers=0, device=0)
        validation_dict[f"round_{r}"] = validation_results

        print("Round", r, "completed")
        clear_output(wait=False)


<h1><b>Define Parameters</b></h1>

In [8]:
#===========================Parameters==============================================================
round_no=30
client_no=4
participating_client=client_no
learning_rate=0.01
batch_size=4
epochs=5
opti='SGD'
# momentum=0.937
# weight_decay=0.0005
data_size=[]
#=====================result variables===============================================================
# iid_model_deviation=[]
# iid_model_deviation.append(0)

#===========other variables=============================================
validation_dict = {}
# # average_weights = {}
# # Define the destination folder
# dst_folder = "Fed_Adagrade_Opt"
# delete_folder(dst_folder)

fl_a="hFL"
set_up="IID"

# set_up="limited_data"

if set_up=="IID":
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
else:
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
    data_size.append(43)
    

forname=set_up

# average_weights = {}
# Define the destination folder
# dst_folder = "Fed_Prox"
# delete_folder(dst_folder)

dst_folder = f"{fl_a}_{forname}_Fed_adagrad_{learning_rate}_{opti}"
delete_folder(dst_folder)

#===================================loading the saved weight list====================================================
global_model = YOLO("initial_weights.pt").to(device)
global_model.info()
initial_weights = {k: v.clone() for k, v in global_model.state_dict().items()}#global_model.state_dict()
print(len(initial_weights))
Print("Model's initial weights", initial_weights)
# global_model.save('current.pt')

Folder 'hFL_IID_Fed_adagrad_0.01_SGD' deleted successfully!
YOLO11n-obb summary: 344 layers, 2,662,287 parameters, 0 gradients, 6.7 GFLOPs
541
Model's initial weights:model.model.0.conv.weight: tensor([[-0.0955, -0.4358, -0.4158],
        [-0.3618, -0.8042, -0.8950],
        [-0.3513, -0.8271, -0.8511]], device='cuda:0')



<h1><b>Round 0</b></h1>

In [9]:
l_model = YOLO("initial_weights.pt").to(device)
#server validation rounds
validation_results = l_model.val(data=f"{fl_a}/c5.yaml", project=f"{dst_folder}/val/round_0", imgsz=512, batch=4,split='val', workers=0,device=0)
validation_dict["round_0"] = validation_results
print(validation_results)

#=================================================================client_1====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c1.yaml", project=f"{dst_folder}/train/round_0_client_1", imgsz=512, batch=4, split='train',  workers=0,device=0)

#=================================================================client_2====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c2.yaml", project=f"{dst_folder}/train/round_0_client_2", imgsz=512, batch=4, split='train',  workers=0,device=0)


#=================================================================client_3====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c3.yaml", project=f"{dst_folder}/train/round_0_client_3", imgsz=512, batch=4, split='train',  workers=0,device=0)


#=================================================================client_4====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c4.yaml", project=f"{dst_folder}/train/round_0_client_4", imgsz=512, batch=4, split='train',  workers=0,device=0)
clear_output(wait=False)

<h1><b>Run FL</b></h1>

In [10]:
# test_model = YOLO("test.pt").to(device)
# # global_model = YOLO("initial_weights.pt").to(device)
# v_results = global_model.val(data=f"{fl_a}/c5.yaml", project=f"{dst_folder}/val/round_0", imgsz=512, batch=4,split='val', workers=0,device=0)
# # validation_dict["round_0"] = validation_results
# print(v_results)


In [11]:
#parameters 3,085,440 parameters, 3,085,424 gradients
federated_learning(initial_weights, client_no, participating_client, round_no, epochs, batch_size)

<h1><b>Save the validation dict</b></h1>

In [12]:
# Convert the dict to a serializable format
def dict_to_serializable(d):
    serializable_dict = {}
    for key, value in d.items():
        if isinstance(value, (int, float, str, list, dict)):
            serializable_dict[key] = value
        else:
            serializable_dict[key] = str(value)  # Convert non-serializable types to string
    return serializable_dict

# Save as JSON
save_dir = dst_folder
os.makedirs(save_dir, exist_ok=True)
file_path = os.path.join(save_dir, 'validation_dict.json')

with open(file_path, 'w') as f:
    json.dump(dict_to_serializable(validation_dict), f, indent=4)

print(f"Validation dictionary saved to {file_path}")


Validation dictionary saved to hFL_IID_Fed_adagrad_0.01_SGD/validation_dict.json


In [13]:
file_path = os.path.join(save_dir, 'validation_dict.json')

# Load the JSON file
with open(file_path, 'r') as f:
    loaded_dict = json.load(f)

# Print the loaded dictionary
print("Validation dictionary loaded successfully")


Validation dictionary loaded successfully


In [14]:
validation_dict['round_20']

ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79dafc062540>
curves: []
curves_results: []
fitness: 0.3530324613399719
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.15566,     0.48315,     0.19842,     0.46327])
names: {0: 'harbor', 1: 'plane', 2: 'swimming-pool', 3: 'tennis-court'}
plot: True
results_dict: {'metrics/precision(B)': 0.5680101252802823, 'metrics/recall(B)': 0.5964888735379301, 'metrics/mAP50(B)': 0.60420825111913, 'metrics/mAP50-95(B)': 0.3251240402533988, 'fitness': 0.3530324613399719}
save_dir: PosixPath('hFL_IID_Fed_adagrad_0.01_SGD/val/round_20/val')
speed: {'preprocess': 0.08460521697998047, 'inference': 3.764524459838867, 'loss': 0.0012254714965820312, 'postprocess': 62.47321128845215}

In [15]:
validation_dict['round_30']

ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79dac411ea50>
curves: []
curves_results: []
fitness: 0.3820037655164941
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.19093,     0.53365,     0.19322,     0.49269])
names: {0: 'harbor', 1: 'plane', 2: 'swimming-pool', 3: 'tennis-court'}
plot: True
results_dict: {'metrics/precision(B)': 0.5934542112967225, 'metrics/recall(B)': 0.6405948654055583, 'metrics/mAP50(B)': 0.646426144977197, 'metrics/mAP50-95(B)': 0.3526235011319716, 'fitness': 0.3820037655164941}
save_dir: PosixPath('hFL_IID_Fed_adagrad_0.01_SGD/val/round_30/val')
speed: {'preprocess': 0.08585453033447266, 'inference': 2.838306427001953, 'loss': 0.0012159347534179688, 'postprocess': 49.81415271759033}

In [16]:
#===========================Parameters==============================================================
round_no=30
client_no=4
participating_client=client_no
learning_rate=0.0001
batch_size=4
epochs=5
opti='SGD'
# momentum=0.937
# weight_decay=0.0005
data_size=[]
#=====================result variables===============================================================
# iid_model_deviation=[]
# iid_model_deviation.append(0)

#===========other variables=============================================
validation_dict = {}
# # average_weights = {}
# # Define the destination folder
# dst_folder = "Fed_Adagrade_Opt"
# delete_folder(dst_folder)

fl_a="hFL"
set_up="IID"

# set_up="limited_data"

if set_up=="IID":
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
else:
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
    data_size.append(43)
    

forname=set_up

# average_weights = {}
# Define the destination folder
# dst_folder = "Fed_Prox"
# delete_folder(dst_folder)

dst_folder = f"{fl_a}_{forname}_Fed_adagrad_{learning_rate}_{opti}"
delete_folder(dst_folder)

#===================================loading the saved weight list====================================================
global_model = YOLO("initial_weights.pt").to(device)
global_model.info()
initial_weights = {k: v.clone() for k, v in global_model.state_dict().items()}#global_model.state_dict()
print(len(initial_weights))
Print("Model's initial weights", initial_weights)
# global_model.save('current.pt')


l_model = YOLO("initial_weights.pt").to(device)
#server validation rounds
validation_results = l_model.val(data=f"{fl_a}/c5.yaml", project=f"{dst_folder}/val/round_0", imgsz=512, batch=4,split='val', workers=0,device=0)
validation_dict["round_0"] = validation_results
print(validation_results)

#=================================================================client_1====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c1.yaml", project=f"{dst_folder}/train/round_0_client_1", imgsz=512, batch=4, split='train',  workers=0,device=0)

#=================================================================client_2====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c2.yaml", project=f"{dst_folder}/train/round_0_client_2", imgsz=512, batch=4, split='train',  workers=0,device=0)


#=================================================================client_3====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c3.yaml", project=f"{dst_folder}/train/round_0_client_3", imgsz=512, batch=4, split='train',  workers=0,device=0)


#=================================================================client_4====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c4.yaml", project=f"{dst_folder}/train/round_0_client_4", imgsz=512, batch=4, split='train',  workers=0,device=0)
clear_output(wait=False)


#parameters 3,085,440 parameters, 3,085,424 gradients
federated_learning(initial_weights, client_no, participating_client, round_no, epochs, batch_size)

# Convert the dict to a serializable format
def dict_to_serializable(d):
    serializable_dict = {}
    for key, value in d.items():
        if isinstance(value, (int, float, str, list, dict)):
            serializable_dict[key] = value
        else:
            serializable_dict[key] = str(value)  # Convert non-serializable types to string
    return serializable_dict

# Save as JSON
save_dir = dst_folder
os.makedirs(save_dir, exist_ok=True)
file_path = os.path.join(save_dir, 'validation_dict.json')

with open(file_path, 'w') as f:
    json.dump(dict_to_serializable(validation_dict), f, indent=4)

print(f"Validation dictionary saved to {file_path}")


file_path = os.path.join(save_dir, 'validation_dict.json')

# Load the JSON file
with open(file_path, 'r') as f:
    loaded_dict = json.load(f)

# Print the loaded dictionary
print("Validation dictionary loaded successfully")


Validation dictionary saved to hFL_IID_Fed_adagrad_0.0001_SGD/validation_dict.json
Validation dictionary loaded successfully


In [17]:
#===========================Parameters==============================================================
round_no=30
client_no=4
participating_client=client_no
learning_rate=0.01
batch_size=4
epochs=5
opti='SGD'
# momentum=0.937
# weight_decay=0.0005
data_size=[]
#=====================result variables===============================================================
# iid_model_deviation=[]
# iid_model_deviation.append(0)

#===========other variables=============================================
validation_dict = {}
# # average_weights = {}
# # Define the destination folder
# dst_folder = "Fed_Adagrade_Opt"
# delete_folder(dst_folder)

fl_a="hFL"
# set_up="IID"

set_up="limited_data"

if set_up=="IID":
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
else:
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
    data_size.append(43)
    

forname=set_up

# average_weights = {}
# Define the destination folder
# dst_folder = "Fed_Prox"
# delete_folder(dst_folder)

dst_folder = f"{fl_a}_{forname}_Fed_adagrad_{learning_rate}_{opti}"
delete_folder(dst_folder)

#===================================loading the saved weight list====================================================
global_model = YOLO("initial_weights.pt").to(device)
global_model.info()
initial_weights = {k: v.clone() for k, v in global_model.state_dict().items()}#global_model.state_dict()
print(len(initial_weights))
Print("Model's initial weights", initial_weights)
# global_model.save('current.pt')


l_model = YOLO("initial_weights.pt").to(device)
#server validation rounds
validation_results = l_model.val(data=f"{fl_a}/c5.yaml", project=f"{dst_folder}/val/round_0", imgsz=512, batch=4,split='val', workers=0,device=0)
validation_dict["round_0"] = validation_results
print(validation_results)

#=================================================================client_1====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c1.yaml", project=f"{dst_folder}/train/round_0_client_1", imgsz=512, batch=4, split='train',  workers=0,device=0)

#=================================================================client_2====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c2.yaml", project=f"{dst_folder}/train/round_0_client_2", imgsz=512, batch=4, split='train',  workers=0,device=0)


#=================================================================client_3====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c3.yaml", project=f"{dst_folder}/train/round_0_client_3", imgsz=512, batch=4, split='train',  workers=0,device=0)


#=================================================================client_4====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c4.yaml", project=f"{dst_folder}/train/round_0_client_4", imgsz=512, batch=4, split='train',  workers=0,device=0)
clear_output(wait=False)


#parameters 3,085,440 parameters, 3,085,424 gradients
federated_learning(initial_weights, client_no, participating_client, round_no, epochs, batch_size)

# Convert the dict to a serializable format
def dict_to_serializable(d):
    serializable_dict = {}
    for key, value in d.items():
        if isinstance(value, (int, float, str, list, dict)):
            serializable_dict[key] = value
        else:
            serializable_dict[key] = str(value)  # Convert non-serializable types to string
    return serializable_dict

# Save as JSON
save_dir = dst_folder
os.makedirs(save_dir, exist_ok=True)
file_path = os.path.join(save_dir, 'validation_dict.json')

with open(file_path, 'w') as f:
    json.dump(dict_to_serializable(validation_dict), f, indent=4)

print(f"Validation dictionary saved to {file_path}")


file_path = os.path.join(save_dir, 'validation_dict.json')

# Load the JSON file
with open(file_path, 'r') as f:
    loaded_dict = json.load(f)

# Print the loaded dictionary
print("Validation dictionary loaded successfully")


Validation dictionary saved to hFL_limited_data_Fed_adagrad_0.01_SGD/validation_dict.json
Validation dictionary loaded successfully


In [18]:
#===========================Parameters==============================================================
round_no=30
client_no=4
participating_client=client_no
learning_rate=0.0001
batch_size=4
epochs=5
opti='SGD'
# momentum=0.937
# weight_decay=0.0005
data_size=[]
#=====================result variables===============================================================
# iid_model_deviation=[]
# iid_model_deviation.append(0)

#===========other variables=============================================
validation_dict = {}
# # average_weights = {}
# # Define the destination folder
# dst_folder = "Fed_Adagrade_Opt"
# delete_folder(dst_folder)

fl_a="hFL"
# set_up="IID"

set_up="limited_data"

if set_up=="IID":
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
else:
    data_size.append(120)
    data_size.append(120)
    data_size.append(120)
    data_size.append(43)
    

forname=set_up

# average_weights = {}
# Define the destination folder
# dst_folder = "Fed_Prox"
# delete_folder(dst_folder)

dst_folder = f"{fl_a}_{forname}_Fed_adagrad_{learning_rate}_{opti}"
delete_folder(dst_folder)

#===================================loading the saved weight list====================================================
global_model = YOLO("initial_weights.pt").to(device)
global_model.info()
initial_weights = {k: v.clone() for k, v in global_model.state_dict().items()}#global_model.state_dict()
print(len(initial_weights))
Print("Model's initial weights", initial_weights)
# global_model.save('current.pt')


l_model = YOLO("initial_weights.pt").to(device)
#server validation rounds
validation_results = l_model.val(data=f"{fl_a}/c5.yaml", project=f"{dst_folder}/val/round_0", imgsz=512, batch=4,split='val', workers=0,device=0)
validation_dict["round_0"] = validation_results
print(validation_results)

#=================================================================client_1====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c1.yaml", project=f"{dst_folder}/train/round_0_client_1", imgsz=512, batch=4, split='train',  workers=0,device=0)

#=================================================================client_2====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c2.yaml", project=f"{dst_folder}/train/round_0_client_2", imgsz=512, batch=4, split='train',  workers=0,device=0)


#=================================================================client_3====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c3.yaml", project=f"{dst_folder}/train/round_0_client_3", imgsz=512, batch=4, split='train',  workers=0,device=0)


#=================================================================client_4====================
l_model = YOLO("initial_weights.pt").to(device)
l_model.val(data=f"{fl_a}/{set_up}/c4.yaml", project=f"{dst_folder}/train/round_0_client_4", imgsz=512, batch=4, split='train',  workers=0,device=0)
clear_output(wait=False)


#parameters 3,085,440 parameters, 3,085,424 gradients
federated_learning(initial_weights, client_no, participating_client, round_no, epochs, batch_size)

# Convert the dict to a serializable format
def dict_to_serializable(d):
    serializable_dict = {}
    for key, value in d.items():
        if isinstance(value, (int, float, str, list, dict)):
            serializable_dict[key] = value
        else:
            serializable_dict[key] = str(value)  # Convert non-serializable types to string
    return serializable_dict

# Save as JSON
save_dir = dst_folder
os.makedirs(save_dir, exist_ok=True)
file_path = os.path.join(save_dir, 'validation_dict.json')

with open(file_path, 'w') as f:
    json.dump(dict_to_serializable(validation_dict), f, indent=4)

print(f"Validation dictionary saved to {file_path}")


file_path = os.path.join(save_dir, 'validation_dict.json')

# Load the JSON file
with open(file_path, 'r') as f:
    loaded_dict = json.load(f)

# Print the loaded dictionary
print("Validation dictionary loaded successfully")


Validation dictionary saved to hFL_limited_data_Fed_adagrad_0.0001_SGD/validation_dict.json
Validation dictionary loaded successfully
